In [16]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# ===== 1. Gerar dados =====
np.random.seed(0)
n_points = 1000
X_data = np.random.uniform(-1.5, 1.5, (n_points, 2))
radii = np.linalg.norm(X_data, axis=1)

def sigmoid(x): return 1 / (1 + np.exp(-x))
sigma_r = .01
probs = sigmoid((1 - radii) / sigma_r)
Y_data = np.random.binomial(1, probs).astype(np.float32)

X_tensor = torch.tensor(X_data, dtype=torch.float32)
Y_tensor = torch.tensor(Y_data.reshape(-1, 1), dtype=torch.float32)

# ===== 2. Definir modelo simples =====
model = nn.Sequential(
    nn.Linear(2, 16),
    nn.ReLU(),
    nn.Linear(16, 16),
    nn.ReLU(),
    nn.Linear(16, 1),
    nn.Sigmoid()
)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# ===== 3. Criar grid para visualização =====
grid_size = 1000
x_vals = np.linspace(-1.5, 1.5, grid_size)
y_vals = np.linspace(-1.5, 1.5, grid_size)
xx, yy = np.meshgrid(x_vals, y_vals)
grid_points = np.c_[xx.ravel(), yy.ravel()]
X_grid = torch.tensor(grid_points, dtype=torch.float32)

# ===== 4. Treinamento por mini-batches (sem DataLoader) =====
batch_size = 64
n_epochs = 10
loss_history = []
loss_history_test = []
outputs_per_batch = []

cnt = 0
for epoch in range(n_epochs):
    indices = np.random.permutation(n_points)
    for start in range(0, n_points, batch_size):
        cnt = cnt+1
        end = start + batch_size
        batch_idx = indices[start:end]
        xb = X_tensor[torch.tensor(batch_idx, dtype=torch.long)]
        yb = Y_tensor[torch.tensor(batch_idx, dtype=torch.long)]

        y_pred = model(xb)
        loss = criterion(y_pred, yb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())

        # Visualização após cada batch
        if cnt % 2 == 0:
            with torch.no_grad():
                pred_map = model(X_grid).numpy().reshape(grid_size, grid_size)
                outputs_per_batch.append(pred_map.copy())

# ===== 5. Gráfico da perda por batch =====
plt.figure(figsize=(8, 4))
plt.plot(loss_history, label='Loss por batch')
plt.xlabel('Batch')
plt.ylabel('Erro')
plt.title('Erro durante o treinamento por mini-batches')
plt.grid(True)
plt.legend()
plt.show()


RuntimeError: Numpy is not available